In [52]:
#############################################
# Kural Tabanlı Sınıflandırma ile Potansiyel Müşteri Getirisi Hesaplama
#############################################

#############################################
# Gezinomi yaptığı satışların bazı özelliklerini kullanarak seviye tabanlı (level based) yeni satış tanımları
# oluşturmak ve bu yeni satış tanımlarına göre segmentler oluşturup bu segmentlere göre yeni gelebilecek müşterilerin şirkete
# ortalama ne kadar kazandırabileceğini tahmin etmek istemektedir.
# Örneğin: Antalya’dan Herşey Dahil bir otele yoğun bir dönemde gitmek isteyen bir müşterinin ortalama ne kadar kazandırabileceği belirlenmek isteniyor.
#############################################

In [53]:
import numpy as np
import pandas as pd
import seaborn as sns

In [54]:
gezinomi = pd.read_excel("miuul_gezinomi.xlsx")
df = gezinomi.copy()
df.head()


,SaleId,SaleDate,CheckInDate,Price,ConceptName,SaleCityName,CInDay,SaleCheckInDayDiff,Seasons
0,415122,2022-12-03,2022-12-03,79.30,Herşey Dahil,Antalya,Saturday,0,Low
1,415103,2022-12-03,2022-12-03,45.97,Yarım Pansiyon,Antalya,Saturday,0,Low
2,404034,2022-09-12,2022-09-13,77.84,Herşey Dahil,Antalya,Tuesday,1,High
3,415094,2022-12-03,2022-12-10,222.71,Yarım Pansiyon,İzmir,Saturday,7,Low
4,414951,2022-12-01,2022-12-03,140.48,Yarım Pansiyon,İzmir,Saturday,2,Low


In [55]:
## Çıktıları düzgün görebilmek için yapılan ayarlar 

pd.set_option("display.max_columns", None)

pd.set_option('display.float_format', lambda x: '%.2f' % x)

In [56]:
#veri seti hk. ön bilgier

df.head()
df.tail()
df.describe()
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 59164 entries, 0 to 59163
Data columns (total 9 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   SaleId              59164 non-null  int64         
 1   SaleDate            59164 non-null  datetime64[ns]
 2   CheckInDate         59164 non-null  datetime64[ns]
 3   Price               59151 non-null  float64       
 4   ConceptName         59164 non-null  object        
 5   SaleCityName        59164 non-null  object        
 6   CInDay              59164 non-null  object        
 7   SaleCheckInDayDiff  59164 non-null  int64         
 8   Seasons             59164 non-null  object        
dtypes: datetime64[ns](2), float64(1), int64(2), object(4)
memory usage: 4.1+ MB


In [57]:
#Kaç unique şehir vardır? Frekansları nedir?

df["SaleCityName"].nunique()   ##number unique değerler 
df["SaleCityName"].value_counts()    ##value counts = değer sayıları = frekanslar

## en çok ziyaret edilen şehir Antalya

Antalya    31649
Muğla      10662
Aydın      10646
Diğer       3245
İzmir       2507
Girne        455
Name: SaleCityName, dtype: int64

In [58]:
#Kaç unique Concept vardır? Ne kadar Satmıştır?

df["ConceptName"].nunique() #eşsiz değerler 
df["ConceptName"].value_counts()    #değer sayıları

## en çok terch. edilen konsept Her Şey dahil

Herşey Dahil      53186
Yarım Pansiyon     3559
Oda + Kahvaltı     2419
Name: ConceptName, dtype: int64

In [59]:
#Şehirlere göre satışlardan toplam ne kadar kazanılmış?

df.groupby("SaleCityName").agg({"Price": "sum"})

#şehirleri ve fiyatları grupla, bu grubun toplamını al
## en çok kazandıran şehir antalya 

,Price
SaleCityName,
Antalya,2041911.10
Aydın,573296.01
Diğer,154572.29
Girne,27065.03
Muğla,665842.21
İzmir,165934.83


In [60]:
#Konseptlere göre satışlardan toplam ne kadar kazanılmış? ve ortalaması neymiş?

df.groupby("ConceptName").agg({"Price" : ["sum","mean"]})

#konseptleri ve fiyatları grupla, bu grubun toplamını ve ort. al

Price      
                      sum  mean
ConceptName                    
Herşey Dahil   3332910.77 62.67
Oda + Kahvaltı  121308.35 50.25
Yarım Pansiyon  174402.35 49.03

In [61]:
#Şehirler ve konseptlerin çaprazlaması (birleşimi) --- birlikte ele alınması

df.groupby(by=["SaleCityName", 'ConceptName']).agg({"Price": ["sum","mean"]})

Price      
                                   sum  mean
SaleCityName ConceptName                    
Antalya      Herşey Dahil   1986734.42 64.52
             Oda + Kahvaltı   37975.88 63.50
             Yarım Pansiyon   17200.80 67.19
Aydın        Herşey Dahil    571056.06 54.00
             Oda + Kahvaltı    1309.43 34.46
             Yarım Pansiyon     930.51 30.02
Diğer        Herşey Dahil     42809.40 84.77
             Oda + Kahvaltı   28462.50 37.60
             Yarım Pansiyon   83300.38 42.11
Girne        Herşey Dahil     10354.18 97.68
             Oda + Kahvaltı    5528.80 39.78
             Yarım Pansiyon   11182.05 53.25
Muğla        Herşey Dahil    618793.90 63.02
             Oda + Kahvaltı   38610.70 59.04
             Yarım Pansiyon    8437.61 45.12
İzmir        Herşey Dahil    103162.80 74.70
             Oda + Kahvaltı    9421.04 41.32
             Yarım Pansiyon   53351.00 59.61

In [62]:
#object degsk.ni kategorik degsk. çevirmek

## aralıkları belirle ve bu aralıkları ata
bins = [-1, 7, 30, 90, df["SaleCheckInDayDiff"].max()]  ## 1-7-30-90 gün ve en max güne göre bins yap yani ambarla, grupla, sınıfla
labels = ["Last Minuters", "Potential Planners", "Planners", "Early Bookers"]  ##bu aralıkları etiketle yani isimlendir

df["EB_Score"] = pd.cut(df["SaleCheckInDayDiff"], bins, labels=labels)  ##EB_Score adnda yeni bir degsk. olustur ve SaleCheckInDayDiff degsk. kesip buraya ata 
df.head(50).to_excel("eb_scorew.xlsx", index=False)  ##buradaki EB_Score degsk. excel formatına degistir ve "eb_scorew.xlsx" olarak kaydet 



In [63]:
#Şehir,Concept, [EB_Score,Sezon,CInday] kırılımında ücret ortalamalarına ve frekanslarına bakınız

# Şehir-Concept-EB Score kırılımında ücret ortalamaları
df.groupby(by=["SaleCityName", 'ConceptName', "EB_Score" ]).agg({"Price": ["mean", "count"]})

# Şehir-Concept-Sezon kırılımında ücret ortalamaları
df.groupby(by=["SaleCityName", "ConceptName", "Seasons"]).agg({"Price": ["mean", "count"]})

# Şehir-Concept-CInday kırılımında ücret ortalamaları
df.groupby(by=["SaleCityName", "ConceptName", "CInDay"]).agg({"Price": ["mean", "count"]})

## Böyle şeyler yaparak görülmeyeni görebiliriz
### Fark etmedigimiz bir şeyi fark edebilir veya başka açıdan bakabiliriz

Price      
                                       mean count
SaleCityName ConceptName    CInDay               
Antalya      Herşey Dahil   Friday    62.66  4136
                            Monday    63.26  6831
                            Saturday  64.42  4741
                            Sunday    65.85  3818
                            Thursday  62.89  3898
...                                     ...   ...
İzmir        Yarım Pansiyon Saturday  52.50   315
                            Sunday    98.93    68
                            Thursday  52.48    91
                            Tuesday   56.14    73
                            Wednesday 54.06    86

[126 rows x 2 columns]

In [64]:
# City-Concept-Season kırılımın çıktısını PRICE'a göre sıralayınız.
## Önceki sorudaki çıktıyı daha iyi görebilmek için sort_values metodunu azalan olacak şekilde PRICE'a uygulayınız.
### Çıktıyı agg_df olarak kaydediniz.

agg_df = df.groupby(["SaleCityName", "ConceptName", "Seasons"]).agg({"Price": "sum"}).sort_values("Price", ascending=False)
agg_df.head(20)



,,,Price
SaleCityName,ConceptName,Seasons,
Antalya,Herşey Dahil,High,1761021.67
Muğla,Herşey Dahil,High,607345.83
Aydın,Herşey Dahil,High,555125.09
Antalya,Herşey Dahil,Low,225712.75
İzmir,Herşey Dahil,High,92239.48
Diğer,Yarım Pansiyon,Low,62798.72
İzmir,Yarım Pansiyon,Low,43007.44
Muğla,Oda + Kahvaltı,High,36268.70
Diğer,Herşey Dahil,High,30498.57


In [65]:
#############################################
#Indekste yer alan isimleri değişken ismine çeviriniz.
#############################################
# Üçüncü sorunun çıktısında yer alan PRICE dışındaki tüm değişkenler index isimleridir.

# Bu isimleri değişken isimlerine çeviriniz.

# İpucu: reset_index()
agg_df.reset_index(inplace=True)
agg_df.head()

,SaleCityName,ConceptName,Seasons,Price
0,Antalya,Herşey Dahil,High,1761021.67
1,Muğla,Herşey Dahil,High,607345.83
2,Aydın,Herşey Dahil,High,555125.09
3,Antalya,Herşey Dahil,Low,225712.75
4,İzmir,Herşey Dahil,High,92239.48


In [66]:
# GÖREV 6: Yeni level based satışları tanımlayınız ve veri setine değişken olarak ekleyiniz.
## sales_level_based adında bir değişken tanımlayınız ve veri setine bu değişkeni ekleyiniz.

agg_df['sales_level_based'] = agg_df[["SaleCityName", "ConceptName", "Seasons"]].agg(lambda x: '_'.join(x).upper(), axis=1)

# sales level based diye bir degsk olusturduk
## bu degskene kırılımları ekledik ve birlestirdk

agg_df.head()

,SaleCityName,ConceptName,Seasons,Price,sales_level_based
0,Antalya,Herşey Dahil,High,1761021.67,ANTALYA_HERŞEY DAHIL_HIGH
1,Muğla,Herşey Dahil,High,607345.83,MUĞLA_HERŞEY DAHIL_HIGH
2,Aydın,Herşey Dahil,High,555125.09,AYDIN_HERŞEY DAHIL_HIGH
3,Antalya,Herşey Dahil,Low,225712.75,ANTALYA_HERŞEY DAHIL_LOW
4,İzmir,Herşey Dahil,High,92239.48,İZMIR_HERŞEY DAHIL_HIGH


In [72]:
#Personaları segmentlere ayırınız.
#############################################

# PRICE'a göre segmentlere ayırınız,
# segmentleri "SEGMENT" isimlendirmesi ile agg_df'e ekleyiniz
agg_df["SEGMENT"] = pd.qcut(agg_df["Price"], 4, labels=["D", "C", "B", "A"])


# segmentleri betimleyiniz
agg_df.head(30)
agg_df.groupby("SEGMENT").agg({"Price": ["mean", "max", "sum"]})


Price                      
             mean        max        sum
SEGMENT                                
D         1675.03    4637.73   15075.30
C         6886.08   10343.56   61974.69
B        15283.69   20501.67  137553.24
A       379335.36 1761021.67 3414018.24

In [74]:
###########################################
# SON OLARAK Oluşan son df'i price değişkenine göre sıralayınız.
#############################################

agg_df.sort_values(by="Price")

# new user adında bir degsk olusturarak personayı gösterin 
## "ANTALYA_HERŞEY DAHIL_HIGH" hangi segmenttedir ve ne kadar ücret beklenmektedir?
new_user = "ANTALYA_HERŞEY DAHIL_HIGH"  

agg_df[agg_df["sales_level_based"] == new_user] #new_user'in atamasını yap

## sales_level_Based = satıs sevys. dayalı olarak


,SaleCityName,ConceptName,Seasons,Price,sales_level_based,SEGMENT
0,Antalya,Herşey Dahil,High,1761021.67,ANTALYA_HERŞEY DAHIL_HIGH,A
